# Tutorial for Introduction to ML Lecture

version 0.1, September 2023

Bryan Scott, CIERA/Northwestern

## Problem 1: Bayes Classifiers

A good starting point for Machine Learning is the Bayes classifier. The basic idea is to assign the most probable label to each data point using Bayes theorem, we take:

$$
p(y | x_n) \propto p(y)p(x_i, ..., x_n | y)
$$

where y is a label for a data point and the $x_n$ are the features of the data that we want to use to classify each data point. A $\textit{Naive} Bayes$ classifier makes an important simplifying assumptions that gives it the name - it assumes that the conditional probabilities are independent, $p(x_i, ..., x_n | y) = p(x_i|y)... p(x_n | y)$. That is, the probability of observing any individual feature doesn't depend on any of the other features. Our task is to construct this classifier from a set of examples we've observed previously and compare it to new data. 

### Part 0: Load and split the data

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

In [29]:
lsst_data = pd.read_csv('simulated_extragalactic_data.csv', index_col=0)

In [30]:
lsst_data[0:1000].to_csv('session_19_DC2_subset.csv', index=False)

In [31]:
lsst_data.shape[0]

1000

In [32]:
lsst_data

,id,flux_g,flux_i,flux_r,flux_u,flux_y,flux_z,truth_type
48682,11567264019,40.96230,36.2869,37.3803,42.79600,58.9711,40.2802,1
40607,11690845043,357.10800,571.4560,471.1240,484.44300,1199.9800,1002.8600,1
22854,11631772147,15.59330,14.6773,14.6509,15.88500,17.0612,14.5978,1
7554,11581148087,51.71180,64.1272,69.5459,22.81470,66.9363,58.9267,1
28188,11631115337,46.70340,42.7023,42.6977,48.35350,58.1880,42.2114,1
...,...,...,...,...,...,...,...,...
36892,11700048639,12.46040,13.9937,12.1656,6.85085,19.1662,17.1776,1
14582,11562625549,31.56970,96.4406,75.8143,11.79810,123.7010,111.2390,1
23836,11627559538,503.87300,942.4960,635.2230,588.65200,1740.1700,1526.6400,1
49128,11565382553,8.78313,15.1979,10.5897,8.62231,29.4473,27.0750,1


### Loading and splitting the data. 

Read in the data, then start by selecting the id, fluxes, and object truth type in the lsst data file you've been provided. 

Once you have selected those, randomly split the data into two arrays, one containing 80% of the data, and a second array containing 20% of the data. 

In [63]:
lsst_data = pd.read_csv('session_19_DC2_subset.csv').reset_index() #path to your data

col_to_classify = ['id','flux_g','flux_i','flux_r','flux_u','flux_y','flux_z','truth_type']

lsst_data_to_classify = lsst_data.loc[:,col_to_classify]
N_data_to_classify = lsst_data_to_classify.shape[0]
random_data = np.sort(np.random.choice(N_data_to_classify, int(N_data_to_classify*0.2), replace=False))

train_data = lsst_data_to_classify.drop(random_data).reset_index(drop=True)
test_data = lsst_data_to_classify.loc[random_data].reset_index(drop=True)

In [64]:
print(random_data)

[  2  16  27  33  34  38  43  48  49  50  51  56  68  71  73  77  80  83
  87  93  94  99 100 104 108 110 121 133 135 155 156 159 163 165 167 171
 173 184 190 192 196 199 204 209 211 215 224 242 270 274 276 282 286 289
 300 306 308 310 314 319 320 322 325 326 327 329 331 333 339 353 355 356
 368 381 385 387 388 398 403 407 415 418 427 431 432 433 434 441 447 454
 455 456 459 460 462 467 477 494 500 501 518 520 530 532 535 546 551 553
 560 561 567 568 572 578 583 585 588 589 595 607 610 612 620 621 626 628
 632 635 640 642 644 650 652 656 665 667 671 673 692 700 703 704 724 726
 730 742 744 752 754 770 775 792 797 798 802 805 807 809 813 818 822 823
 830 831 835 836 838 845 849 861 863 871 881 883 885 891 898 903 904 908
 913 915 920 921 927 929 935 939 948 950 952 954 967 972 978 979 980 982
 991 999]


In [65]:
test_data

,id,flux_g,flux_i,flux_r,flux_u,flux_y,flux_z,truth_type
0,11631772147,15.59330,14.6773,14.65090,15.88500,17.0612,14.5978,1
1,11565452667,27.71430,26.5303,25.52000,30.37460,37.5289,38.3631,1
2,40745568334,65544.80000,92359.0000,84819.30000,28672.20000,94089.5000,93977.1000,2
3,11639300481,146.39600,109.8540,129.27900,94.26480,136.1850,125.9980,1
4,11568919652,11.59090,13.2037,10.40450,11.27840,22.1336,14.2701,1
...,...,...,...,...,...,...,...,...
195,11567862080,8.48342,11.6592,9.07956,8.58720,26.3974,14.5710,1
196,11575651179,9.28551,12.8492,11.75760,3.42997,18.5936,15.7811,1
197,11575481781,15.27300,15.5485,14.85870,10.42210,18.3602,17.4043,1
198,11626280998,5.20117,22.9789,10.79200,3.24677,27.9421,25.8164,1


In [66]:
train_data

,id,flux_g,flux_i,flux_r,flux_u,flux_y,flux_z,truth_type
0,11567264019,40.96230,36.2869,37.3803,42.79600,58.9711,40.2802,1
1,11690845043,357.10800,571.4560,471.1240,484.44300,1199.9800,1002.8600,1
2,11581148087,51.71180,64.1272,69.5459,22.81470,66.9363,58.9267,1
3,11631115337,46.70340,42.7023,42.6977,48.35350,58.1880,42.2114,1
4,11690275081,118.30800,373.8760,211.8320,120.56900,959.9060,784.7030,1
...,...,...,...,...,...,...,...,...
795,11694913248,68.07290,74.1691,58.9424,50.73660,94.5738,86.5480,1
796,11700048639,12.46040,13.9937,12.1656,6.85085,19.1662,17.1776,1
797,11562625549,31.56970,96.4406,75.8143,11.79810,123.7010,111.2390,1
798,11627559538,503.87300,942.4960,635.2230,588.65200,1740.1700,1526.6400,1


### Part 1: Estimate Class Frequency in the training set

One of the ingredients in our classifier is p(y), the unconditional class probabilities. 

We can get this by counting the number of rows belonging to each class in train_data and dividing by the length of the training data set. 

In [67]:
def estimate_class_probabilities(data, class_column_name):
    """
    Computes unconditional class probabilities. 
     
    Args:
        x_train (array): training data for the classifier
 
    Returns:
        ints p1, p2: unconditional probability of an element of the training set belonging to class 1
    """
    
    p1 = data.loc[data[class_column_name] == 1].shape[0]/data.shape[0]
    p2 = data.loc[data[class_column_name] == 2].shape[0]/data.shape[0]

    # print(data.loc[data[class_column_name] == 2].shape[0])
    # print(data.loc[data[class_column_name] == 1].shape[0])
    # print(data.shape[0])

    return p1, p2

p1, p2 = estimate_class_probabilities(train_data, 'truth_type')

In [68]:
p1, p2

(0.82625, 0.17125)

### Part 2:  Feature Likelihoods

We are assuming that the relationship between the classes and feature probabilities are related via:

$p(x_i, ..., x_n | y) =  p(x_i|y)... p(x_n | y)$

however, we still need to make an assumption about the functional form of the $p(x_i | y)$. As a simple case, we will assume $p(x_i | y)$ follows a Gaussian distribution given by:

$$
p(x_i | y) = \frac{1}{\sqrt{2 \pi \sigma_y}} \exp{\left(-\frac{(x_i - \mu)^2}{\sigma_y^2}\right)}
$$

and we will make a maximum likelihood estimate of $\mu$ and $\sigma_y$ from the data. This means using empirical estimates $\bar{x}$ and $\hat{\sigma}$ as estimators of the true parameters $\mu$ and $\sigma_y$. 

Write a fitting function that takes the log of the fluxes and returns an estimate of the parameters of the per-feature likelihoods for each class.

In [69]:
col_fluxes = ['flux_g','flux_i','flux_r','flux_u','flux_y','flux_z']

In [40]:
np.mean(train_data[col_fluxes], axis=0)

flux_g    231500.334667
flux_i    443167.480949
flux_r    366335.958190
flux_u     75500.916188
flux_y    497683.143453
flux_z    480758.377767
dtype: float64

In [70]:
def per_feature_likelihood_parameters(x_train, label):
    """"
    Computes MAP estimates for the class conditional likelihood. 
     
    Args:
        x_train (array or pd series): training data for the classifier
        label (int): training labels for the classifier 
 
    Returns:
        means, stdevs (array): MAP estimates of the Gaussian conditional probability distributions for a specific class
    """
    
    means = np.mean(np.log10(x_train[label]), axis=0)
    stdevs = np.std(np.log10(x_train[label]), axis=0)
    
    return means, stdevs


In [71]:
def likelihood(x, mean, stdev):
    """"
    Computes the likelihood of a data point x under a Gaussian distribution with parameters mean and stdev. 
     
    Args:
        x (float): data point (assume already in log scale)
        mean (float): mean of the Gaussian distribution
        stdev (float): standard deviation of the Gaussian distribution
 
    Returns:
        likelihood (float): likelihood of x under the Gaussian distribution
    """
    
    likelihood = norm.pdf(x, loc=mean, scale=stdev)
    
    return likelihood

### Part 3: MAP Estimates of the Class Probabilities

Now that we have the unconditional class probabilities and the parameters of the per feature likelihoods in hand, we can put this all together to build the classifier. Use the methods you have already written to write a function that takes in the training data and returns fit parameters. Once you have done that, write a method that takes the fit parameters as an argument and predicts the class of new (and unseen) data. 

In [72]:
# build the classifier

# solved 

def fit(x_train):
    """"
    Convenience function to perform fitting on the training data
     
    Args:
        x_train (array or pd series): training data for the classifier
 
    Returns:
        p1, p2, class_1_mean, class_2_mean, class_1_std, class_2_std: see documentation for per_feature_likelihood_parameters
    """
    
    # compute probabilities and MAP estimates of the Gaussian distribution's parameters using the methods you wrote above
    p1, p2 = estimate_class_probabilities(x_train, 'truth_type')
    class_1_mean, class_1_std = per_feature_likelihood_parameters(x_train.loc[x_train['truth_type']==1], col_fluxes)
    class_2_mean, class_2_std = per_feature_likelihood_parameters(x_train.loc[x_train['truth_type']==2], col_fluxes)

    return p1, p2, class_1_mean, class_2_mean, class_1_std, class_2_std


In [73]:
p1, p2, class_1_mean, class_2_mean, class_1_std, class_2_std = fit(train_data)

In [85]:

def predict(x_test, class_probability, class_means, class_dev, class_column_name):
    """"
    Predict method
     
    Args:
        x_test (array): data to perform classification on
        class_probability (array): unconditional class probabilities
        class_means, class_dev (array): MAP estimates produced by the fit method
 
    Returns:
        predict_List (list): class membership predictions
    """
    
    # compute probabilities of an element of the test set belonging to class 1 or 2
    predict_list = np.zeros(x_test.shape[0])

    for i in range(x_test.shape[0]):
        prior = class_probability
        likelihoods = np.prod([likelihood(np.log10(x_test.loc[i,class_column_name[j]]), class_means[j], class_dev[j]) for j in range(len(class_means))])
        posterior = prior*likelihoods
        predict_list[i] = posterior
    
    
    return predict_list

In [86]:
train_data.loc[2,col_fluxes[0]]

51.7118

In [87]:
prob_1 = predict(test_data, p1, class_1_mean, class_1_std, col_fluxes)
prob_2 = predict(test_data, p2, class_2_mean, class_2_std, col_fluxes)

In [88]:
prob_1

array([2.33462875e-02, 6.48637360e-02, 3.42620654e-45, 3.13572561e-03,
       1.72255723e-02, 1.31723109e-04, 2.04062289e-02, 6.08253901e-03,
       3.22519981e-02, 1.23634640e-04, 7.78188814e-02, 1.53719053e-02,
       2.42700412e-05, 8.98531611e-03, 3.51007984e-02, 2.34222232e-02,
       3.77597547e-02, 3.67869375e-03, 3.18991338e-05, 1.77534305e-02,
       7.69377689e-02, 4.22649095e-02, 2.01625444e-02, 2.04631835e-02,
       2.93705536e-08, 2.67071984e-02, 6.22691924e-03, 2.62142686e-02,
       5.13343166e-02, 3.97498661e-25, 2.83491454e-02, 5.64363286e-02,
       5.84848479e-02, 5.92732083e-03, 3.44762951e-02, 5.84443922e-02,
       9.58800149e-61, 1.07228104e-35, 5.80478079e-03, 6.48432473e-04,
       8.16328917e-02, 4.40130355e-02, 2.63328477e-02, 6.92323586e-02,
       1.07695274e-05, 4.31333528e-02, 2.16200437e-26, 6.39115292e-02,
       2.21094483e-02, 2.64938384e-33, 5.57425910e-03, 6.88258673e-02,
       4.06713556e-02, 5.17114205e-02, 4.26936014e-03, 2.38104621e-03,
      

In [89]:
predict_label = np.zeros(test_data.shape[0])

In [90]:
for i in range(len(test_data)):
    if prob_1[i] > prob_2[i]:
        predict_label[i] = 1
    else:
        predict_label[i] = 2
    

In [91]:
predict_label

array([1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 2., 2., 1., 1., 1., 1., 1., 1., 2., 1., 2., 1., 1., 2., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 2., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       1., 1., 1., 1., 2., 1., 2., 1., 1., 1., 2., 2., 1., 2., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 2., 2., 1., 1.,
       1., 2., 1., 1., 2., 2., 1., 2., 1., 1., 2., 1., 1., 2., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.])

### Part 4: Metrics

After creating a classifier, you now want to evaluate it in terms of how often it correctly and incorrectly classifies the objects in your training set. To do this, we'll design a confusion matrix. A confusion matrix is a matrix whose entries are the counts of the predicted vs actual class. For example, the first entry is the count of objects that are predicted to be of class 1 and actually are of class 1 and so on, while the off-diagonal elements would be instances of class 1 that are predicted to be of class 2, and instances of class 2 that are predicted to be of class 1. 

In [37]:
def plot_confusion_matrix(df_confusion, cmap=):
    """
    
    Convenience function to plot the confusion matrix from a pd.crosstab object. Hint: use plt.matshow and choose a sensible color map.
    
    Args:
        df_confusion (pd.crosstab): A pd.crosstab object.
        
    Returns:
        null 
    """
    
    
    plt.matshow()


## Problem 2: The Cramer-Rao bound (pen & paper, challenging, optional)

As we saw in the lecture, the Cramer-Rao bound is an important result in statistics that has intuitive consequences for many applied problems in ML. The proof of the Cramer-Rao bound can be insightful to work through. 

The starting point for the proof of the bound is the Cauchy-Schwarz inequality, which can be used to show that:

$$
[Cov(U, V)]^2 \le Var(U)Var(V)
$$

Starting from the definitions that U = T(X), where T(X) is an estimator of some parameter $\theta$ of the distribution $f(X|\theta)$ from which the data is sampled, and V = $\frac{\partial}{\partial \theta} log f(X |\theta)$. Use the Cauchy-Schwarz inequality to show the Cramer-Rao bound for these choices of U and V. 

$\textit{Hint:}$ you will need the fact that the $\mathbb{E}(V) = 0$, where $\mathbb{E}$ is the expectation of a random variable.